# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,batangafo,7.3008,18.2833,33.68,18,90,0.79,CF,1710266253
1,1,mar del plata,-38.0023,-57.5575,21.40,93,75,6.71,AR,1710265971
2,2,blackmans bay,-43.0167,147.3167,10.29,81,59,1.05,AU,1710266255
3,3,rundu,-17.9333,19.7667,30.00,21,59,1.52,NaN,1710266257
4,4,gobabis,-22.4500,18.9667,26.86,30,77,2.78,NaN,1710266258


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display


# Configure the map plot
cities_worldwide_map = city_data_df.hvplot.points("Lng",                   # longitude of each city
                                                  "Lat",                   # Latitude of each city
                                                  geo = True,              # Turns map into a geographic map 
                                                  tiles = "OSM",           # Defines the "tiles" ==> OSM stands for "OpenStreetMap" which gives us the intituve looking map below (white = land) & (blue = water)
                                                  size="Humidity",         # Makes the size of the plotted dots based on the humidity of each city where the nearest hotel is located
                                                  alpha = .5,              # Alpha = .5 is making the colors of the dots more transparent so they arent so opaque making it easier to see where each hotel is located on the map
                                                  color = "City",          # Assigning different colors for each city and puts the cities as a list/legend on the right side of the map
                                                  frame_width = 1000 ,     # Makes the map output larger horizontally (width) so its easier to visualization
                                                  frame_height = 500)      # Makes the map output larger vertically (height) so its easier to visualization

# Display the map
cities_worldwide_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

# Filtering "city_data_df" to a new dataframe called "specific_criteria_df" which includes 1) A max temperature less than 35 degrees celsius (95 degrees fahrenheit) but greater than 15 degrees celsius (59 degrees fahrenheit)
#                                                                                          2) Wind speed less than 4.5 m/s
#                                                                                          3) Zero cloudiness

specific_city_criteria_df = city_data_df[(city_data_df['Max Temp'] < 35) & (city_data_df['Max Temp'] > 15) &
                                    (city_data_df['Wind Speed'] < 4.5) & 
                                    (city_data_df['Cloudiness'] == 0)]



# Drop any rows with null values            
specific_city_criteria_df.dropna()                 # .dropna() will only drop rows with null values if a row / "city" is pulled in with a null value in any of the columns to clean the dataframe and make it complete

# Display sample data
specific_city_criteria_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,farshut,26.0549,32.1633,24.29,9,0,3.56,EG,1710266287
88,88,illapel,-31.6308,-71.1653,28.06,29,0,3.56,CL,1710266171
106,106,mount isa,-20.7333,139.5000,20.87,73,0,3.09,AU,1710266415
110,110,red wing,44.5625,-92.5338,17.28,33,0,4.12,US,1710266272
145,145,fort myers beach,26.4520,-81.9481,28.36,59,0,2.06,US,1710266496
180,180,fort bragg,35.1390,-79.0060,22.58,20,0,2.57,US,1710266562
199,199,brookings,44.3114,-96.7984,15.81,41,0,1.54,US,1710266339
304,304,chinchilla,-26.7500,150.6333,16.12,90,0,3.81,AU,1710266760
335,335,geraldton,-28.7667,114.6000,22.23,83,0,4.12,AU,1710266955
344,344,tura,25.5198,90.2201,20.12,34,0,2.04,IN,1710267213


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = specific_city_criteria_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

#Adding an empty column for "Hotel Name" to store the hotel found using Geoapify API
hotel_df['Hotel Name']=''

#Adding the empty column "Hotel Name" into the dataframe "hotel_df"
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name']]

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
24,farshut,EG,26.0549,32.1633,9,
88,illapel,CL,-31.6308,-71.1653,29,
106,mount isa,AU,-20.7333,139.5000,73,
110,red wing,US,44.5625,-92.5338,33,
145,fort myers beach,US,26.4520,-81.9481,59,
180,fort bragg,US,35.1390,-79.0060,20,
199,brookings,US,44.3114,-96.7984,41,
304,chinchilla,AU,-26.7500,150.6333,90,
335,geraldton,AU,-28.7667,114.6000,83,
344,tura,IN,25.5198,90.2201,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000                                # Setting the radius to "10,000 meters" / about 6.2 miles to find the first hotel located within that radius of each city
params = {
    "categories" : "accommodation.hotel",     # "accommodation.hotel" is from the geoapify.com documentation to only pull hotel names
    "apiKey" : geoapify_key,                  # Defining the geoapify_key to pull data from geoapify.com 
    "limit" : 20                              # Setting limit to 20 so that the API key only pulls 20 times and prevents over usages of apikey to avoid getting locked out for 24 hours
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():                             ## For "each" index in "each" row in hotel_df ==> '.interrows' function interates through each row (start of for loop)
    # get latitude, longitude from the DataFrame
    lat = row['Lat']                                              # "lat" varible is assigned to each row at the column "Lat" in the hotel_df dataframe
    lng = row['Lng']                                              # "lng" varible is assigned to each row at the column "Lng" in the hotel_df dataframe
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'                             # Creating filter param to find the hotels within a CIRCLE radius of 10000 meters based on a given cities Lat & Long in the hotel_df dataframe
    params["bias"] = f'proximity:{lng},{lat}'                                     # Creating bias param to tell the code to find just the proximity of a hotel within a given cities Lat & Long in the hotel_df dataframe (hence => Closest hotel)
    
    # Set base URL ==> This base url is define on the geoapify.com website
    base_url = "https://api.geoapify.com/v2/places"                      


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)     # Setting up API request for finding hotels 
    
    # Convert the API response to JSON format
    name_address = name_address.json()                    ## Turning the geoapify request that pulls in the hotel data into json data
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]    ## If a hotel is found for that city => The name of that hotel is found based on "filtering" through the json data
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"                                ## If no hotel is found for a given city in hotel_df==> The code wont error out and will set that city equal to "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")        # Creating an f-string to print out "city" - nearest hotel: "hotel name" ONLY IF a hotel was found for that given city

# Display sample data
hotel_df

Starting hotel search
farshut - nearest hotel: Hotel Central
illapel - nearest hotel: Domingo Ortiz de Rosas
mount isa - nearest hotel: Ibis Styles
red wing - nearest hotel: The Saint James Hotel
fort myers beach - nearest hotel: Key Estero Shops
fort bragg - nearest hotel: Airborne Inn Lodging
brookings - nearest hotel: Quality Inn
chinchilla - nearest hotel: Chinchilla Downtown Motor Inn
geraldton - nearest hotel: Ocean Centre Hotel
tura - nearest hotel: Hotel Polo Orchid
ar rutbah - nearest hotel: No hotel found
mawlaik - nearest hotel: No hotel found
jaisalmer - nearest hotel: Jeet Mahal
seminole - nearest hotel: Best Western Plus Sanford
mallawi - nearest hotel: No hotel found
bou hanifia el hamamat - nearest hotel: Station thermale منتجع
east florence - nearest hotel: Residence Inn
ramhormoz - nearest hotel: No hotel found
puerto escondido - nearest hotel: Hotel Barlovento


,City,Country,Lat,Lng,Humidity,Hotel Name
24,farshut,EG,26.0549,32.1633,9,Hotel Central
88,illapel,CL,-31.6308,-71.1653,29,Domingo Ortiz de Rosas
106,mount isa,AU,-20.7333,139.5000,73,Ibis Styles
110,red wing,US,44.5625,-92.5338,33,The Saint James Hotel
145,fort myers beach,US,26.4520,-81.9481,59,Key Estero Shops
180,fort bragg,US,35.1390,-79.0060,20,Airborne Inn Lodging
199,brookings,US,44.3114,-96.7984,41,Quality Inn
304,chinchilla,AU,-26.7500,150.6333,90,Chinchilla Downtown Motor Inn
335,geraldton,AU,-28.7667,114.6000,83,Ocean Centre Hotel
344,tura,IN,25.5198,90.2201,34,Hotel Polo Orchid


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot

                                        
hotel_vacation_map = hotel_df.hvplot.points("Lng",                                   # longitude of each city
                                            "Lat",                                   # Latitude of each city
                                            geo = True,                              # Turns map into a geographic map 
                                            tiles = "OSM",                           # Defines the "tiles" ==> OSM stands for "OpenStreetMap" which gives us the intituve looking map below (white = land) & (blue = water)
                                            size="Humidity",                         # Makes the size of the plotted dots based on the humidity of each city where the nearest hotel is located
                                            scale = 1.5,                             # Scale = 1.5 is just increasing the size of the dots on the map for easier visualization
                                            alpha = .5,                              # Alpha = .5 is making the colors of the dots more transparent so they arent so opaque making it easier to see where each hotel is located on the map
                                            color = "City",                          # Asigning different colors for each city and puts the cities as a list/legend on the right side of the map
                                            hover_cols=["Hotel Name","Country"],     # Defines what other infomation that should be shown when you hover over a plot on the graph 
                                            frame_width = 1000,                      # Makes the map output larger horizontally (width) so its easier to visualization
                                            frame_height = 500)                      # Makes the map output larger vertically (height) so its easier to visualization

# Display the map
hotel_vacation_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)